In [1]:
import requests as rq
import bs4 as bs4
import time
import json
import pandas as pd
import numpy as np

# Parte 1: Editoras

In [2]:
# Url onde estão todas as editoras

url_editoras = 'https://www.skoob.com.br/editoras/'

In [8]:
# Fazendo request do link e transformando em um objeto do bs4

response = rq.get(url_editoras)
pagina_html = bs4.BeautifulSoup(response.text)

In [29]:
# Procurando tags

tags = pagina_html.findAll(id="resultadoBusca-")
editoras = tags[0].findAll('strong')

In [73]:
# Criando dataframe que armazenará os dados

df = pd.DataFrame()

In [ ]:
# Loop em todas as editoras para que seja extraído o nome e o link
# Os dados serão salvos em um JSON

for x in editoras:
    if x.find('a') != None:
        editora = x.get_text()
        link = x.find('a').get('href')
        with open('editoras.json', 'a+') as output:
            data = {'editora': editora, 'link': link}
            output.write('{}\n'.format(json.dumps(data)))   

In [86]:
df_editoras = pd.read_json('editoras.json', lines=True)

In [87]:
df_editoras

,editora,link
0,Intrínseca,/editora/11-intrinseca
1,Sextante,/editora/19-sextante
2,Arqueiro,/editora/21-arqueiro
3,Galera Record,/editora/25-galera-record
4,Rocco,/editora/41-rocco
5,Valentina,/editora/64-valentina
6,Editora Planeta,/editora/67-editora-planeta
7,Verus,/editora/98-verus
8,HarperCollins Brasil,/editora/100-harpercollins-brasil
9,Universo dos Livros,/editora/102-universo-dos-livros


# Parte 2: Livros das Editoras


Os livros de cada editora estão no link : "https://www.skoob.com.br/editora/livros/codigoDeCadaEditora"  
É necessário o código de cada editora para entrar na página dos livros. Os códigos são os números da coluna link do df_editoras

In [88]:
# Função para ser aplicada em cada linha do dataframe na coluna código
# Parâmetro x: string -> link da editora
# Irá retornar o código da editora

def codigo(x):
    return x.split('/')[-1].split('-')[0]

In [89]:
codigos = df_editoras['link'].apply(codigo)
df_editoras['codigo'] = codigos

In [90]:
df_editoras

,editora,link,codigo
0,Intrínseca,/editora/11-intrinseca,11
1,Sextante,/editora/19-sextante,19
2,Arqueiro,/editora/21-arqueiro,21
3,Galera Record,/editora/25-galera-record,25
4,Rocco,/editora/41-rocco,41
5,Valentina,/editora/64-valentina,64
6,Editora Planeta,/editora/67-editora-planeta,67
7,Verus,/editora/98-verus,98
8,HarperCollins Brasil,/editora/100-harpercollins-brasil,100
9,Universo dos Livros,/editora/102-universo-dos-livros,102


In [25]:
# Salvando dados em um csv

df_editoras.to_csv('editoras_com_código.csv', index=False)

In [26]:
## Editora AllBook

In [48]:
url_livros = 'https://www.skoob.com.br/editora/livros/{}/mpage:{}'

In [51]:
# Coletando as páginas onde são mostrados os livros de cada editora

for x in range(1,3):
    url_formatada = url_livros.format(df_editoras['codigo'][13], x)
    response = rq.get(url_formatada)
    
    with open('./dados_brutos/livros_{}_{}.html'.format(df_editoras['codigo'][13], x), "w+", encoding="utf-8") as output:
        output.write(response.text)

In [91]:
# Coletando o título dos livros e o link

for x in range(1,3):
    with open('./dados_brutos/livros_139_{}.html'.format(x), 'r+', encoding="utf-8") as inp:
        page_html = inp.read()
        parsed = bs4.BeautifulSoup(page_html)
        lista_livros = parsed.findAll('div','box_livro')
        for y in range(0,len(lista_livros)):
            nome = lista_livros[y].find('h3').get_text()
            link = lista_livros[y].find('a').get('href')
            with open('titulo_e_link.json', 'a+') as output:
                        data = {'titulo': nome, 'link': link}
                        output.write('{}\n'.format(json.dumps(data)))

In [116]:
df_aux = pd.read_json('titulo_e_link.json', lines=True)

In [117]:
df_aux

,titulo,link
0,O Manual da Conquista,/o-manual-da-conquista-1256838ed11616124.html
1,Estrada para lugar nenhum,/estrada-para-lugar-nenhum-1074042ed1076839.html
2,Tudo termina aqui,/tudo-termina-aqui-1131815ed1135352.html
3,Ricos Indecentes,/ricos-indecentes-1008042ed1135947.html
4,Um amor em Nova York,/um-amor-em-nova-york-1149088ed1151354.html
5,Mister Jesus,/mister-jesus-1036747ed1154608.html
6,Você é meu oceano,/voce-e-meu-oceano-1169158ed1168741.html
7,Drive,/drive-1176518ed1174848.html
8,A Escolha,/a-escolha-1188657ed1185735.html
9,Além das Bases,/alem-das-bases-1197468ed1193955.html
